# Make Web Site

In [1]:
import sys, os, io, datetime
import json
import random
import requests
import zipfile
import glob
import re
import shutil
import pandas as pd
from slugify import slugify
from tqdm.notebook import tqdm
from html import escape
from collections import defaultdict

In [2]:
import corpi
import cdli
import oracc
import languages

In [3]:
pd.set_option("display.max_columns", None)

In [4]:
supported_langs = set(["akk", "sux"])

In [5]:
wwwroot = os.path.abspath("../dist")
os.makedirs(wwwroot, exist_ok=True)
wwwroot

'/Volumes/home/Projects/CuneiformTranslators/dist'

## Download the Catalog

In [6]:
cat = cdli.get_catalog()

/Volumes/home/Projects/CuneiformTranslators/tools/cdli.py:222: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,14,16,17,19,22,23,24,25,26,27,28,29,31,32,33,37,39,42,45,46,49,52,53,54,55,57,58,59,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  cat = pd.read_csv(io.StringIO(cat_csv))


In [7]:
cat

,accession_no,accounting_period,acquisition_history,alternative_years,ark_number,atf_source,atf_up,author,author_remarks,cdli_collation,cdli_comments,citation,collection,composite_id,condition_description,date_entered,date_of_origin,date_remarks,date_updated,dates_referenced,db_source,designation,dumb,dumb2,electronic_publication,elevation,excavation_no,external_id,findspot_remarks,findspot_square,genre,google_earth_collection,google_earth_provenience,height,id,id_text2,id_text,join_information,language,lineart_up,material,museum_no,object_preservation,object_type,period,period_remarks,photo_up,primary_publication,provenience,provenience_remarks,publication_date,publication_history,published_collation,seal_id,seal_information,stratigraphic_level,subgenre,subgenre_remarks,surface_preservation,text_remarks,thickness,translation_source,width,object_remarks
0,NaN,NaN,NaN,NaN,21198/zz001q0dtm,"Englund, Robert K.",NaN,CDLI,"31x61x18; Lú A 14-16.30-32.48-50; M XVIII, auf...",NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2020-03-14,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 065",NaN,NaN,NaN,NaN,"W 06435,a",NaN,auf Hügeloberfläche in der Nähe des Südbaues,"M XVIII,?",Lexical,NaN,NaN,31,1,0,1,NaN,undetermined,150ppi 20160630,clay,VAT 01533,NaN,tablet,Uruk III (ca. 3200-3000 BC),NaN,NaN,"CDLI Lexical 000002, ex. 065",Uruk (mod. Warka),NaN,2015ff.,"Englund, Robert K. & Nissen, Hans J., ATU 3 (1...",NaN,NaN,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,18,no translation,61,NaN
1,NaN,NaN,NaN,NaN,21198/zz001q0dv4,"Englund, Robert K.",NaN,CDLI,30x48x13; Lú A 13-15.23-25.?; Fundstelle wie W...,NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2018-10-20,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 066",NaN,NaN,NaN,NaN,"W 06435,b",NaN,auf der Hügeloberfläche in der Nähe des Südbaues,"M XVIII,?",Lexical,NaN,NaN,30,2,0,2,NaN,undetermined,150ppi 20160630,clay,VAT 15263,NaN,tablet,Uruk III (ca. 3200-3000 BC),NaN,NaN,"CDLI Lexical 000002, ex. 066",Uruk (mod. Warka),NaN,2015ff.,"Englund, Robert K. & Nissen, Hans J., ATU 3 (1...",NaN,NaN,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,13,no translation,48,NaN
2,NaN,NaN,NaN,NaN,21198/zz001q0dwn,"Englund, Robert K.",NaN,"Englund, Robert K. & Nissen, Hans J.","42x53x19; Vocabulary 9; Qa XVI,2, unter der Ab...",NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",NaN,NaN,12/4/2001,NaN,NaN,2020-01-26,NaN,20011204 protocuneiform_catalogue,"ATU 3, pl. 081, W 9123,d",NaN,NaN,NaN,NaN,"W 09123,d",NaN,"unter der Abgleichung der Schicht III, 1,5 m ü...","Qa XVI,2",Lexical,NaN,NaN,42,3,0,3,NaN,undetermined,150ppi 20160630,clay,VAT 15253,NaN,tablet,Uruk IV (ca. 3350-3200 BC),NaN,NaN,"ATU 3, pl. 081, W 9123,d",Uruk (mod. Warka),NaN,1993,"ATU 1, 539",NaN,NaN,NaN,NaN,Archaic Vocabulary (witness),Text category: 15-09; Foreign ID: LVO 9,NaN,NaN,19,no translation,53,NaN
3,NaN,NaN,NaN,NaN,21198/zz001q0dx5,"Englund, Robert K.",NaN,CDLI,26x23x23; Lú A 9-10.?.?; Fundstelle wie W 9123...,NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2018-10-20,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 051",NaN,NaN,NaN,NaN,"W 09169,d",NaN,"unter der Abgleichung der Schicht III, 1,5 m ü...","Qa XVI,2",Lexical,NaN,NaN,26,4,0,4,NaN,undetermined,150ppi 20160630,clay,VAT 15168,NaN,tablet,Uruk IV (ca. 3350-3200 BC),NaN,NaN,"CDLI Lexical 000002, ex. 051",Uruk (mod. Warka),NaN,2015ff.,"Englund, Robert K. & Nissen, Hans J., ATU 3 (1...",NaN,NaN,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,23,no translation,23,NaN
4,NaN,NaN,NaN,NaN,21198/zz001q0dzp,"Englund, Robert K.",NaN,CDLI,"29x36x20; Lú A Vorläufer; Qa XVI,2, unter der ...",NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2018-10-20,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 172",NaN,NaN,NaN,NaN,"W 09206,k",

In [8]:
len(cat)

353283

In [9]:
cat.columns

Index(['accession_no', 'accounting_period', 'acquisition_history',
       'alternative_years', 'ark_number', 'atf_source', 'atf_up', 'author',
       'author_remarks', 'cdli_collation', 'cdli_comments', 'citation',
       'collection', 'composite_id', 'condition_description', 'date_entered',
       'date_of_origin', 'date_remarks', 'date_updated', 'dates_referenced',
       'db_source', 'designation', 'dumb', 'dumb2', 'electronic_publication',
       'elevation', 'excavation_no', 'external_id', 'findspot_remarks',
       'findspot_square', 'genre', 'google_earth_collection',
       'google_earth_provenience', 'height', 'id', 'id_text2', 'id_text',
       'join_information', 'language', 'lineart_up', 'material', 'museum_no',
       'object_preservation', 'object_type', 'period', 'period_remarks',
       'photo_up', 'primary_publication', 'provenience', 'provenience_remarks',
       'publication_date', 'publication_history', 'published_collation',
       'seal_id', 'seal_information', 's

https://cdli.ucla.edu/search/archival_view.php?ObjectID=P256681

In [10]:
cat[cat["id_text"]==256681]

,accession_no,accounting_period,acquisition_history,alternative_years,ark_number,atf_source,atf_up,author,author_remarks,cdli_collation,cdli_comments,citation,collection,composite_id,condition_description,date_entered,date_of_origin,date_remarks,date_updated,dates_referenced,db_source,designation,dumb,dumb2,electronic_publication,elevation,excavation_no,external_id,findspot_remarks,findspot_square,genre,google_earth_collection,google_earth_provenience,height,id,id_text2,id_text,join_information,language,lineart_up,material,museum_no,object_preservation,object_type,period,period_remarks,photo_up,primary_publication,provenience,provenience_remarks,publication_date,publication_history,published_collation,seal_id,seal_information,stratigraphic_level,subgenre,subgenre_remarks,surface_preservation,text_remarks,thickness,translation_source,width,object_remarks
106189,NaN,NaN,NaN,NaN,21198/zz001s1m1w,"de Ridder, Alba",NaN,"Stol, Marten",NaN,NaN,NaN,NaN,University of Pennsylvania Museum of Archaeolo...,NaN,NaN,2/24/2005,NaN,NaN,2021-06-12,NaN,20050224 fitzgerald_upenn,"AbB 11, 029",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Letter,NaN,NaN,?,106191,0,256681,NaN,Akkadian,NaN,clay,UM 29-16-076,NaN,tablet,Old Babylonian (ca. 1900-1600 BC),NaN,600ppi 20160630,"AbB 11, 029",Nippur (mod. Nuffar),NaN,1986,NaN,NaN,NaN,NaN,NaN,NaN,Letter; 7x8x1 line,NaN,NaN,?,no translation,?,NaN


## Get the Human Transliterations

In [11]:
cdli_corpus = corpi.CDLI()

Parsing atf


In [12]:
print("Merging transliterations with catalog")
cdli_pubs = cdli.merge_atf_with_catalog(cdli_corpus.cdli_pubs, cat, tqdm)

Merging transliterations with catalog


  0%|          | 0/135256 [00:00<?, ?it/s]

In [79]:
oracc_dir="/Volumes/FrankDisk/oracc_zips"
oracc_corpus = corpi.ORACC(oracc_dir=oracc_dir, download=False, tqdm=tqdm)

In [14]:
cdli_pubs[0]

Publication('P000001', 'qpc', [TextArea('tablet', [], []), TextArea('obverse', [], []), TextArea('column 1', [TextLine("1'.", '1(N01) , [...]', {}), TextLine("2'.", '1(N01) , TIM ABGAL#', {}), TextLine("3'.", '1(N01) , KINGAL#', {})], []), TextArea('column 2', [TextLine("1'.", '1(N01) , [...]', {}), TextLine("2'.", '1(N01) , GAL~a# UMUN2#', {}), TextLine("3'.", '1(N01) , GAL~a UMUN2 KU3~a', {})], []), TextArea('column 3', [TextLine("1'.", '1(N01) , DUB~a SANGA~a#', {}), TextLine("2'.", '1(N01) , SUG5# SAG#', {}), TextLine("3'.", '1(N01) , UB SAG#', {})], []), TextArea('reverse', [TextLine('1.', '[N] , [...]', {})], [])])

In [61]:
all_pubs = dict()
for p in oracc_corpus.oracc_transliterated_pubs.values():
    if p.language in supported_langs:
        p.corpus = "oracc"
        all_pubs[p.id] = p
for p in cdli_pubs:
    if p.language in supported_langs:
        p.corpus = "cdli"
        all_pubs[p.id] = p
print(f"Found {len(all_pubs)} unique publications")

Found 136614 unique publications


In [62]:
# Group publications by the first 4 characters of pub.id to create a directory structure
all_pubs_by_dir = defaultdict(list)
for p in all_pubs.values():
    all_pubs_by_dir[p.id[:4].lower()].append(p)
print(f"Found {len(all_pubs_by_dir)} directories")


Found 398 directories


## Get the ML Translations

In [63]:
translations_json_path = "../data/ml_translations.json"
translations = json.loads(str(open(translations_json_path, "rb").read(), "utf8"))
for k in translations.keys():
    if "_to_" in k:
        print(k, len(translations[k]))

akk_to_en 380004
sux_to_en 50000


In [64]:
num_transliterations = len(all_pubs)
num_translations = len([x for x in all_pubs.values() if x.has_translations()])
print("num_transliterations:", num_transliterations)
print("    num_translations:", num_translations)

num_transliterations: 136614
    num_translations: 65529


## Find Publications we have translations for

In [65]:
tgt_lang = "en"

translated_pubs = []

for pub in tqdm(list(all_pubs.values())):
    has_new_translations = False
    has_ml_translations = True
    has_lines = False
    st_key = f"{pub.language}_to_{tgt_lang}"
    if st_key not in translations:
        continue
    st_translations = translations[st_key]
    for a in pub.text_areas:
        if len(a.lines) > 0 and len(a.paragraphs) == 0:
            a.lines_to_paragraphs(pub.language, tgt_lang)
        paras = a.paragraphs_to_lines()
        for i, plines in enumerate(paras):
            p = a.paragraphs[i]
            p.languages["ml_"+tgt_lang] = ""
            head = ""
            for si,ei,s in plines:
                if len(s) > 1:
                    has_lines = True
                    has_new_translations = has_new_translations or (tgt_lang not in p.languages)
                    if s in st_translations:
                        p.languages["ml_"+tgt_lang] += head + st_translations[s]
                        head = " "
                    else:
                        has_ml_translations = False
    pub.has_new_translations = has_new_translations
    pub.has_ml_translations = has_ml_translations
    if has_ml_translations and has_lines:
        translated_pubs.append(pub)
    
newly_translated_pubs = [x for x in translated_pubs if x.has_new_translations]
print(len(translated_pubs), "translated_pubs")
print(len(newly_translated_pubs), "newly_translated_pubs")

  0%|          | 0/136614 [00:00<?, ?it/s]

848 translated_pubs
746 newly_translated_pubs


In [66]:
len([x for x in translated_pubs if x.language == "sux"]), "sux"

(164, 'sux')

In [67]:
newly_translated_pubs[0]

Publication('P226077', 'akk', [TextArea('object cylinder', [], []), TextArea('surface a', [TextLine("1'.", '[...] x [_...]-nita2#_ babila#[{ki} ...]', {}), TextLine("2'.", '[...]-tu s,e-he-ri#-[...] {d#}iszkur# {d}[...] _en#-me_-szu2 it-tak-x [...]-szak#-szi-du-szu2 i#?-[...]', {}), TextLine("3'.", '[...] nu-uh-hu lib3-bi _dingir_-ti#-x [x] u nu-up-pu-usz ka-bat-[...] da#-ru-u2 it-ru-s,u [...]', {}), TextLine("4'.", '[...] x _an-szar2_ {d}5(u) {d}en u _dumu_ {d}en _dingir-mesz#_ ti-ik-li-szu2 kul#-lat# [...] gi#-mir ma-li-ki u2-szak-ni-szu sze#-[...]', {}), TextLine("5'.", '[...]-u2 _e2_ an-szar2 mu-ud-disz e2-kur e-pisz e2-sag-il2# u babila#[{ki} ...]-szak#-lil esz-re-e-ti u ma-ha-zi mu-kin# sat-[...]', {}), TextLine("6'.", '[...] x _bala_-szu2 _en gal_-u {d}marduk a-na babila2{ki} sa-li-me ir#-[...]-il2 _e2-gal_-szu2 ir-mu-u szu-[...]', {}), TextLine("7'.", '[...]-na# _iri_-szu2 bad3-an{ki} u _e2_-szu2 e2-dim-gal-kalam-ma u2-sze-ri#-[...]-bu# pa-rak da-ra-[...]', {}), TextLine("8'.",

## Data Dimensions

In [68]:
browser_dimensions = [
#     ("new", lambda p: ["new" if p.has_new_translations else "old"]),
#     ("language", lambda p: [p.language]),
    ("object_type", lambda p: [cdli.get_object_type(p.object_type)]),
    ("genre", lambda p: cdli.get_genres(p.genre)),
    ("period", lambda p: [cdli.period_slug_from_period[x] for x in cdli.get_periods(p.period)]),
]

## HTML Components

## HTML Pages

In [69]:
def get_file_path(site_path):
    return f"{wwwroot}{site_path}"

def get_page_file_path(site_path):
    return f"{get_file_path(site_path)}.html"
    

In [70]:
style = """
@font-face {
    font-family: "textfont";
    src: url("/fonts/cmunorm.woff") format('woff');
}
@font-face {
    font-family: "textfont";
    src: url("/fonts/cmunoti.woff") format('woff');
    font-style: italic;
}
@font-face {
    font-family: "headingfont";
    src: url("/fonts/cmunss.woff") format('woff');
}
@font-face {
    font-family: "headingfont";
    src: url("/fonts/cmunsx.woff") format('woff');
    font-weight: bold;
}

body { font-family: headingfont; }
div.content { margin: 0 auto; max-width: 2048px; }
h1,h2,h3,h4,h5,h6 { font-family: headingfont; }
h1 { text-align: center; }
p.src { font-size: 80%; font-style: italic;}
.otitle { text-align: center; }
h1.otitle { margin-top: 1em; padding-top: 1em; border-top:solid 2px rgba(128,128,128,0.5); }
.line:hover, .line-match { background-color: rgba(255, 255, 128, 1.0); }
.langid {
    font-family: textfont;
    font-style: italic;
    text-align: center;
    font-size: 14pt;
}
.lang-akkts, .lang-akk p, .lang-akk li, 
.lang-suxts, .lang-sux p, .lang-sux li {
    font-style: italic;
    opacity: 0.75;
    font-size: 11pt !important;
    line-height: 12pt !important;
}
.lang-en { opacity: 0.5; }
.lang-ml_en {}
.translations-container {
    display: flex;
    flex-flow: row wrap;
    justify-content: center;
    align-items: flex-start;
    align-content: flex-start;
    margin-bottom: 2em;
}
.text {
    line-height: 14pt;
    font-family: textfont;
    max-width: 400px;
    margin: 0em 1em;
    font-size: 12pt;
}
.textarea h1 {
    font-family: textfont;
    font-variant: small-caps;
    font-size: 18pt;
    margin-top: 2em;
}
"""

In [71]:
script = """
const lines = document.querySelectorAll('.line');
for (let $line of lines) {
    let matchedElements = [];
    $line.addEventListener("mouseenter", function( event ) {
        let p = event.target.parentElement;
        while (p != null && !p.classList.contains('translations-container')) {
            p = p.parentElement;
        }        
        if (p != null) {
            let linen = "line-";
            for (let c of event.target.classList) {
                if (c.startsWith('line-')) {
                    linen = c + "";
                }
            }
            function highlight(e) {
                if (e === event.target)
                    return;
                if (e.classList.contains(linen)) {
                    e.classList.add('line-match');
                    matchedElements.push(e);
                }
                else {
                    for (let c of e.children) {
                        highlight(c);
                    }
                }
            }
            highlight(p);
        }
        
    });
    $line.addEventListener("mouseleave", function( event ) {
        for (let e of matchedElements) {
            e.classList.remove('line-match');
        }
    });
}
"""

In [72]:
def header(title, f):
    f.write(f"<DOCTYPE html>\n")
    f.write(f"<html>\n<head>\n")
    f.write(f"<meta charset='utf-8'>\n")
    f.write(f"<title>{escape(title)}</title>\n")
    f.write(f"<meta name='viewport' content='width=device-width, initial-scale=1'>\n<style>{style}</style></head>\n")
    f.write(f"<body>\n<div class='content'><h1>{escape(title)}</h1>\n")
    f.write(f"<p class='otitle'>ML Translations by <a href=\"https://huggingface.co/praeclarum/cuneiform\">praeclarum/cuneiform</a></p>\n")
    
def footer(f):
    f.write(f"</div>\n")
    f.write(f"<script>\n")
    f.write(script)
    f.write(f"</script>\n")
    f.write(f"</body>\n</html>")

In [73]:
def start_page(path, title):
    file_path = get_page_file_path(path)
    file_dir = os.path.dirname(file_path)
    os.makedirs(file_dir, exist_ok=True)
    f = open(file_path, "wt")
#     print(f"Writing {path} at {file_path}")
    header(title, f)
    return f

def end_page(f):
    footer(f)

### Publication Index Pages

In [36]:
data_links = [
    ("Oracc - Open Richly Annotated Cuneiform Corpus", "http://oracc.museum.upenn.edu"),
    ("CDLI - Cuneiform Digital Library Initiative", "https://cdli.ucla.edu"),
    ("ETCSL - Electronic Text Corpus of Sumerian Literature", "https://etcsl.orinst.ox.ac.uk"),
]

In [74]:
language_sort = {
    "akk": 0,
#     "akkts": 1,
#     "elx": 2,
#     "elxts": 3,
    "sux": 4,
#     "suxts": 5,
    "ml_en": 100,
    "en": 1000,
#     "fr": 1001,
}

bad_translators = {"uncertain", "NaN", "no translation", "", "check"}

In [80]:
oracc_pub_ids_and_langs, transliterated_oracc_corpi = oracc.load_all_project_pub_ids(oracc_dir, tqdm=tqdm)
# transliterated_oracc_corpi = oracc_corpus.transliterated_oracc_corpi

  0%|          | 0/140 [00:00<?, ?it/s]

Error: (<class 'json.decoder.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x7fcc9130c100>)
Error: (<class 'json.decoder.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x7fcaf44cf500>)
Error: (<class 'json.decoder.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x7fcad11da200>)
Error: (<class 'json.decoder.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x7fcb86fad040>)
Error: (<class 'json.decoder.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x7fcba6152dc0>)
Error: (<class 'json.decoder.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x7fcb13c69a00>)
Error: (<class 'json.decoder.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <t

In [81]:
def paragraphs_to_html(a, paragraphs, text_lines):
    html = []
    for pi, plines in enumerate(paragraphs):
        p = a.paragraphs[pi]
        tag = p.tag
        html.append(f"<{tag}>")
        for line_start_index, line_end_index, text in plines:
            for i in range(line_start_index, line_end_index):
                if i >= len(text_lines):
                    continue
                html.append(f"<span class='line line-{i}'>{escape(text_lines[i])}</span>\n")
        html.append(f"</{tag}>\n")
    return "".join(html)

def title_case(str):
    if len(str) == 0:
        return str
    if len(str) == 1:
        return str.upper()
    return str[0].upper() + str[1:]

def output_pub(p, f):
    f.write(f"<h1 class='otitle'>{p.id}: {' and '.join(cdli.get_genres(p.genre))} {cdli.get_object_type(p.object_type)}</h1>\n")
    src_a = ""
    if p.corpus == "cdli":
        src_a = f"<a href='https://cdli.ucla.edu/search/archival_view.php?ObjectID={p.id}'>CDLI</a>"
    elif p.corpus == "oracc":
        oracc_project_id = transliterated_oracc_corpi[p.id]["corpus"]["project"]
        src_a = f"<a href='http://oracc.museum.upenn.edu/{oracc_project_id}/corpus'>Oracc</a>"
    f.write(f"<p class='otitle'>{p.period} {src_a}</p>\n")
    areas_with_lines = [x for x in p.text_areas if any(x.lines)]
    for a in areas_with_lines:
        f.write(f"<section class='textarea'>\n")
        if len(areas_with_lines) > 1:
            f.write(f"<h1>{escape(title_case(a.name))}</h1>\n")
        f.write(f"<div class='translations-container'>\n")
        paragraphs = a.paragraphs_to_lines()
        texts = {p.language: paragraphs_to_html(a, paragraphs, [l.text for l in a.lines])}
        langs = set()
        for para in a.paragraphs:
            for lang in para.languages:
                if lang in language_sort:
                    langs.add(lang)
        for lang in langs:
            texts[lang] = paragraphs_to_html(a, paragraphs, [l.languages[lang] if lang in l.languages else " " for l in a.lines])
        langs.add(p.language)
#             f.write(f"<p><pre>{escape(repr(paragraphs))}</pre></p> ")
#             if "akkts" in langs and "akk" in langs:
#                 langs.remove("akk")
#             if "suxts" in langs and "sux" in langs:
#                 langs.remove("sux")
        langs = sorted(list(langs), key=lambda x:language_sort[x])
        for lang in langs:
            f.write(f"<div class='lang-{lang} text'>\n")
            translator = "ML Translation" if lang.startswith("ml_") else (languages.all_languages[lang])
            if lang == tgt_lang:
                # if p.translation_source not in bad_translators:
                #     translator = escape(p.translation_source)
                # else:
                translator = "Anonymous"
            f.write(f"<div class='langid'>{translator}.</div>\n")
            f.write(texts[lang])
            f.write(f"</div>\n")
        f.write(f"</div></section>\n")

In [82]:
for pdir in tqdm(list(all_pubs_by_dir.keys())):
    pubs = sorted(all_pubs_by_dir[pdir], key=lambda p:p.id)
    with start_page(f"/p/{pdir}", pdir) as f:
        for p in pubs:
            f.write(f"<section id='{p.id}' class='pub'>\n")
            output_pub(p, f)
            f.write(f"</section>\n")
        end_page(f)

  0%|          | 0/398 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [33]:
def output_browser(path, pubs, ignore_dims, f):
    next_pages = []
    if len(pubs) == 0:
        return next_pages
    f.write(f"<section>\n")
    for dname, dselect in browser_dimensions:
        if dname in ignore_dims:
            continue
        vgroups = defaultdict(lambda: [])
        for p in pubs:
            for v in dselect(p):
                vgroups[v].append(p)
        if len(vgroups) < 2:
            continue
        f.write(f"<h1>{escape(dname)}</h1>\n")    
        for gv in vgroups.keys():
            gpubs = vgroups[gv]
            if len(gpubs) > 0:
                next_pages.append((dname, gv, gpubs))
                f.write(f"<a href='{path}/{gv}/'>{len(gpubs)} {escape(gv)}</a>\n")
    f.write(f"</section>\n")
    f.write(f"<section>\n")
    max_on_page = 200
    if len(pubs) <= max_on_page or len(next_pages) == 0:
        for p in pubs:
            output_pub(p, f)        
    else:
        f.write(f"<p>{len(pubs)} publications. Narrow the list to less than {max_on_page} by choosing links above.</p>\n")
    f.write(f"</section>\n")
    return next_pages

def output_browser_page(path, dim_value, pubs, ignore_dims):
    with start_page(path + "/index", f"{len(pubs)} {escape(dim_value)}") as f:
        next_pages = output_browser(path, pubs, ignore_dims, f)
        end_page(f)
    for gk, gv, gpubs in next_pages:
        ignores = set(ignore_dims)
        ignores.add(gk)
        output_browser_page(f"{path}/{gv}", gv, gpubs, ignores)


In [34]:
os.makedirs(f"{wwwroot}/fonts", exist_ok=True)
for font in glob.glob("../fonts/*.woff"):
    shutil.copy2(font, f"{wwwroot}/fonts/{os.path.basename(font)}")

In [35]:
shutil.copy("../web/translator.html", f"{wwwroot}/translator.html")

'/Volumes/home/Projects/CuneiformTranslators/dist/translator.html'

In [37]:
languages.old_languages

{'akk': 'Akkadian',
 'sux': 'Sumerian',
 'qpn': 'Proper Nouns',
 'arc': 'Aramaic',
 'elx': 'Elamite',
 'grc': 'Greek',
 'peo': 'Old Persian',
 'ug': 'Ugaritic',
 'xur': 'Urartian'}

In [38]:
languages.cuneiform_text_to_unicode("ki na-lu5-ta ur-nigar{gar} szu ba-ti iti masz-da3-gu7 mu us2-sa ki-masz{ki} ba-hul", "sux")

'𒆠 𒈾lu5𒋫 𒌨nigar{𒃻} szu 𒁀𒋾 iti maszda3gu7 𒈬 us2𒊓 𒆠masz{𒆠} 𒁀hul'

In [39]:
languages.cuneiform_text_to_unicode("iti", "sux")

'iti'

In [42]:
#!rm -r /Users/fak/Dropbox/Projects/CuneiformTranslators/dist

In [44]:


with start_page("/404", "Not Found") as f:
    f.write(f"<p>The page you are looking for is not here.</p>\n")
    footer(f)
    
print("Writing /")
with start_page("/index", "ML Cuneiform Translations") as f:
    f.write(f"<a href='/translator'>Online Translator!</a>")
    pubs = translated_pubs
#     pubs = [p for p in output_pubs if p.id>393000 and p.id<394000]
    by_lang = defaultdict(lambda: [])
    for p in pubs:
        by_lang[p.language].append(p)
    f.write(f"<section><h1>language</h1></section>")
    next_pages = []
    for lang in sorted(list(by_lang.keys())):
        gpubs = by_lang[lang]
        f.write(f"<a href='/{lang}/'>{len(gpubs)} {escape(lang)}</a>\n")
        next_pages.append(("language", lang, gpubs))
    f.write(f"</section>")
    next_pages.extend(output_browser("", pubs, "", f))
    footer(f)
    
for gk, gv, gpubs in tqdm(next_pages):
    output_browser_page(f"/{gv}", gv, gpubs, set([gk]))
    
for f in glob.glob("../dist/*"):
    print(f)

Writing /


AttributeError: 'Publication' object has no attribute 'object_type'